In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import functools
import collections

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from UTILS import preprocessing_experiment2, general_cleaning, simple_cleaning_experiment2,verification_based_on_initial_record,\
OneTimeSeriesPlot, ZoneVariable, HenVariable, print_color, boxplot_distribution_entropy, heatmap_duration_perzone_perhen, \
time_series_henColumn_tsRow, cleaning_mouvement_records
import config_experiment2 as config

# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
id_run = config.id_run
dico_matching = config.dico_matching
nbr_sec = config.nbr_sec

change the configuration file if not done yet!


# General cleaning verification

In [4]:
#for being more reproductible, we open the file that was saved from cleaning
df_pr = pd.read_csv(os.path.join(path_extracted_data, id_run+'_PreprocessRecords.csv'), sep=';', parse_dates=['Timestamp'],
                 index_col=0) 
print(df_pr.shape)
df_pr.head(3)

c:\users\camil\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(2744938, 7)


,Timestamp,HenID,Zone,PenID,log_file_name,ts_order,TrackingTag
520693,2019-11-11 00:00:15,5pb,Tier 1,5,Abteile_3_5_log_00000003,54318,54
520694,2019-11-11 00:00:38,11sp,Tier 3 Rampe + Nestbox,11,Abteile_10_12_log_00000004,14461,69
520695,2019-11-11 00:00:44,11sp,Tier 4 + obere Stange,11,Abteile_10_12_log_00000004,14462,69


In [5]:
df_gc = pd.read_csv(os.path.join(path_extracted_data, id_run+'_records_GeneralCleaning.csv'), sep=';', 
                 parse_dates=['Timestamp', 'Timestamp_initial','day']) 
print(df_gc.shape)
df_gc.head(3)

(2744938, 10)


,Timestamp,HenID,Zone,PenID,log_file_name,day,ts_order_logname,ts_order_list,ms,Timestamp_initial
0,2019-11-11 00:00:15,hen_5pb,2 Zone,5,Abteile_3_5_log_00000003,2019-11-11,Abteile_3_5_log_00000003_54318,Abteile_3_5_log_00000003_54318,0.0,2019-11-11 00:00:15
1,2019-11-11 00:00:38,hen_11sp,4 Zone,11,Abteile_10_12_log_00000004,2019-11-11,Abteile_10_12_log_00000004_14461,Abteile_10_12_log_00000004_14461,0.0,2019-11-11 00:00:38
2,2019-11-11 00:00:44,hen_11sp,5 Zone,11,Abteile_10_12_log_00000004,2019-11-11,Abteile_10_12_log_00000004_14462,Abteile_10_12_log_00000004_14462,0.0,2019-11-11 00:00:44


### from the preprocessing file to the general_cleaning file

In [6]:
#still same number of columns
if df_gc.shape[0]!=df_pr.shape[0]:
    print('ERROR: the preprocessing file has not the same number of column as the general_cleaning one')
    sys.exit()

In [7]:
#different columns
print('The new columns generated by the general cleaning are: \n','\n '.join([x for x in df_gc.columns if x not in df_pr.columns]))

The new columns generated by the general cleaning are: 
 day
 ts_order_logname
 ts_order_list
 ms
 Timestamp_initial


### initial_timestamp vs new one

In [8]:
df_gc['nbr_record'] = df_gc['ts_order_list'].map(lambda x: len(x.split(' ')))

In [9]:
df_gc[df_gc['nbr_record']==3][['HenID','Timestamp', 'Timestamp_initial', 'nbr_record', 'ts_order_list']].head(6)

,HenID,Timestamp,Timestamp_initial,nbr_record,ts_order_list
51880,hen_10pb,2019-11-12 05:40:56.000000,2019-11-12 05:40:56,3,Abteile_10_12_log_00000004_37945 Abteile_10_12...
51881,hen_10pb,2019-11-12 05:40:56.333333,2019-11-12 05:40:56,3,Abteile_10_12_log_00000004_37945 Abteile_10_12...
51882,hen_10pb,2019-11-12 05:40:56.666666,2019-11-12 05:40:56,3,Abteile_10_12_log_00000004_37945 Abteile_10_12...
73626,hen_5gs,2019-11-12 13:34:55.000000,2019-11-12 13:34:55,3,Abteile_3_5_log_00000003_92539 Abteile_3_5_log...
73627,hen_5gs,2019-11-12 13:34:55.333333,2019-11-12 13:34:55,3,Abteile_3_5_log_00000003_92539 Abteile_3_5_log...
73628,hen_5gs,2019-11-12 13:34:55.666666,2019-11-12 13:34:55,3,Abteile_3_5_log_00000003_92539 Abteile_3_5_log...


In [10]:
df_gc[df_gc['nbr_record']>3][['HenID', 'Timestamp', 'Timestamp_initial', 'nbr_record', 'ts_order_list']]

,HenID,Timestamp,Timestamp_initial,nbr_record,ts_order_list
1051123,hen_12pp,2019-12-03 14:50:53.000,2019-12-03 14:50:53,4,Abteile_10_12_log_00000009_76948 Abteile_10_12...
1051125,hen_12pp,2019-12-03 14:50:53.250,2019-12-03 14:50:53,4,Abteile_10_12_log_00000009_76948 Abteile_10_12...
1051126,hen_12pp,2019-12-03 14:50:53.500,2019-12-03 14:50:53,4,Abteile_10_12_log_00000009_76948 Abteile_10_12...
1051127,hen_12pp,2019-12-03 14:50:53.750,2019-12-03 14:50:53,4,Abteile_10_12_log_00000009_76948 Abteile_10_12...


# General rule info

In [14]:
#for being more reproductible, we open the file that was saved from cleaning
df_rule = pd.read_csv(os.path.join(path_extracted_data, id_run+'_records_with_some_cleaned_info_3.csv'), 
                      sep=';', parse_dates=['Timestamp']) 
print(df_rule.shape)
df_rule.head(10)

(1150305, 13)


,log_file_name,PenID,Timestamp,HenID,Zone,Zone_without_flickering,is_impossible_mvt,zone_flickering2,enveloppe,flickering2_name,previous_zone_flickering2,is_of_interest,interesting_sequ
0,Abteile_10_12_log_00000004,10,2019-11-11 06:15:11,hen_10bb,4 Zone,4 Zone,NaN,4 Zone,4 Zone,NaN,NaN,False,NaN
1,Abteile_10_12_log_00000004,10,2019-11-11 06:15:27,hen_10bb,5 Zone,5 Zone,NaN,5 Zone,5 Zone,NaN,4 Zone,False,NaN
2,Abteile_10_12_log_00000004,10,2019-11-11 06:23:51,hen_10bb,4 Zone,4 Zone,NaN,4 Zone,4 Zone,NaN,5 Zone,False,NaN
3,Abteile_10_12_log_00000004,10,2019-11-11 06:40:15,hen_10bb,3 Zone,3 Zone,NaN,flickering2,3 Zone,23.0,4 Zone,False,NaN
4,Abteile_10_12_log_00000004,10,2019-11-11 06:51:30,hen_10bb,2 Zone,2 Zone,NaN,flickering2,enveloppe,NaN,flickering2,False,NaN
5,Abteile_10_12_log_00000004,10,2019-11-11 07:11:13,hen_10bb,3 Zone,3 Zone,NaN,flickering2,enveloppe,NaN,flickering2,False,NaN
6,Abteile_10_12_log_00000004,10,2019-11-11 07:11:15,hen_10bb,2 Zone,2 Zone,NaN,flickering2,enveloppe,NaN,flickering2,False,NaN
7,Abteile_10_12_log_00000004,10,2019-11-11 07:20:43,hen_10bb,3 Zone,3 Zone,NaN,flickering2,3 Zone,NaN,flickering2,False,NaN
8,Abteile_10_12_log_00000004,10,2019-11-11 07:22:00,hen_10bb,2 Zone,2 Zone,NaN,flickering2,enveloppe,12.0,flickering2,True,23.0
9,Abteile_10_12_log_00000004,10,2019-11-11 07:27:44,hen_10bb,1 Zone,1 Zone,NaN,flickering2,enveloppe,NaN,flickering2,False,NaN


In [ ]:
#TODO once we have the correct cleaning or once we use this clening info

# Bining verification

#open one bining csv
li_path = glob.glob(os.path.join(path_extracted_data, 'HensTimeSeries', id_run+'_ts_MostFrequentZone_period*.csv'))
ind = 0
dico_mi_li_df = {}
path_ = li_path[ind]
h = 'hen_'+path_.split('hen_')[1][:-4]
if 'VERIFICATION' in path_:
    sys.exit()
period = int(path_.split('period')[1].split('_')[0])
print(h, period)   
df_h = pd.read_csv(path_, sep=';', parse_dates=['New_Timestamp']) 
print(df_h.shape)
df_h.head(3)

In [ ]:
#TODO if we use it, for now its easier to simply open the csv